In [1]:
#!/usr/bin/python2.7
# change keywords in line 82; this script will fecth 3000 tweets
# if you get no module named oauth2, install with pip2.7 install --user oauth2
# the tweets will be written in a file tweets.csv
# make sure you change the outout file name when you run this script multiple timees.
import oauth2 as oauth
import csv
from csv import writer
import json
import time
import urllib2 as urllib
import re

In [11]:
def strip_non_ascii(string):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in string if 0 < ord(c) < 127)
    return ''.join(stripped)

api_key = "****************8"
api_secret = "****************"
access_token_key = "**********************"
access_token_secret = "*********************"

_debug=0

oauth_token    = oauth.Token(key=access_token_key, secret=access_token_secret)
oauth_consumer = oauth.Consumer(key=api_key, secret=api_secret)

signature_method_hmac_sha1 = oauth.SignatureMethod_HMAC_SHA1()

http_method = "GET"

http_handler  = urllib.HTTPHandler(debuglevel=_debug)
https_handler = urllib.HTTPSHandler(debuglevel=_debug)

'''
Construct, sign, and open a twitter request
using the hard-coded credentials above.
'''
def twitterreq(url, method, parameters):
  req = oauth.Request.from_consumer_and_token(oauth_consumer,
                                             token=oauth_token,
                                             http_method=http_method,
                                             http_url=url,
                                             parameters=parameters)

  req.sign_request(signature_method_hmac_sha1, oauth_consumer, oauth_token)

  headers = req.to_header()

  if http_method == "POST":
    encoded_post_data = req.to_postdata()
  else:
    encoded_post_data = None
    url = req.to_url()

  opener = urllib.OpenerDirector()
  opener.add_handler(http_handler)
  opener.add_handler(https_handler)

  response = opener.open(url, encoded_post_data)

  return response

#####get big ben clock tweets id
def fetch_bigben():
  url = "https://api.twitter.com/1.1/search/tweets.json?q=big_ben_clock&count=100&result_type=recent"
  parameters = []
  response = twitterreq(url, "GET", parameters)
  return response

id_list=[]
re=fetch_bigben()
re_j=json.load(re)
re_j=re_j['statuses']
for a in re_j:
  id_list.append(a["id"])
#####end of big ben

#####get tweets about a topic
tweets_num=300 # number of tweets we want
parameters = []
js=[]
for i in range(tweets_num/100):
  url = "https://api.twitter.com/1.1/search/tweets.json?q='Tesla'&lang=en&count=100&result_type=recent&max_id="+str(id_list[2*i])
  response = twitterreq(url, "GET", parameters)
  re_j=json.load(response)
  re_j=re_j['statuses']
  js.append(re_j)
  time.sleep(1)

ids=[]
screen_name=[]
followers=[]
listed=[]
retweet=[]
inreplyto=[]
favorite=[]
friends=[]
text=[]
location=[]
following_count=[]
mentions_received=[]
retweets_received=[]
follower_count=[]
listed_count=[]
mentions_sent=[]
retweets_sent=[]
posts=[]
for tweets in js:
  for tweet in tweets:
    if tweet.get('user'):
        ids.append(tweet['user']['id'])
        retweet.append(tweet['retweet_count'])
        favorite.append(tweet['favorite_count'])
        inreplyto.append(tweet['in_reply_to_screen_name'])
        following_count.append(tweet['user']['friends_count'])
        screen_name.append(tweet['user']['screen_name'])
        follower_count.append(tweet['user']['followers_count'])
        listed_count.append(tweet['user']['listed_count'])
        text.append(tweet['text'])
        location.append(tweet['user']['location'])
        #mentions_received.append(tweet['mentions'])
        #retweets_received.append(tweet['user']['retweets_received'])
        #mentions_sent.append(tweet['user']['mentions_sent'])
        #retweets_sent.append(tweet['user']['retweets_sent'])
        posts.append(tweet['user']['statuses_count'])
####################################################

#######output tweets file
out = open('tweets_2_2.csv','wb')
print >> out, 'ids, screen_name, follower_count, retweet, inreplyto, favorite, following_count, listed_count, location, text'

rows = zip(ids, screen_name, follower_count, retweet, inreplyto, favorite, following_count, listed_count, location, text)

csv = writer(out)

for row in rows:
    values = [strip_non_ascii(str(value.encode('utf8'))) if hasattr(value, 'encode') else strip_non_ascii(str(value)) for value in row]
    csv.writerow(values)
print("Written to tweets_tesla_1.csv in Files")
out.close()

Written to tweets_tesla_1.csv in Files


In [ ]:
path ='D:\UT_Austin\Spring\social media analytics\HW1\Twitter_data'

files_in_dir = [f for f in os.listdir(path) if f.endswith('csv')]
for filenames in files_in_dir:
   df = pd.read_csv(path +"\\" + filenames)
   df.to_csv('out.csv', mode='a')